In [5]:
# Cell 1 — Imports & paths
import os
import numpy as np
import pandas as pd

FEATURES_PATH = "/Users/jaidevreddy/Documents/Documents - Jaidev’s MacBook Pro/projects/blr-renewables/data/processed/phase4_features.parquet"
TARGET_PATH   = "/Users/jaidevreddy/Documents/Documents - Jaidev’s MacBook Pro/projects/blr-renewables/data/processed/daily_energy.parquet"
OUT_DATASET   = "data/processed/phase5_dataset.parquet"
OUT_BASELINES = "data/processed/phase5_baselines.csv"

os.makedirs("data/processed", exist_ok=True)


In [6]:
# Cell 2 — Load data & quick inspect
X = pd.read_parquet(FEATURES_PATH)   # should contain: date, ZoneID, [feature columns...]
y = pd.read_parquet(TARGET_PATH)     # should contain: date, ZoneID, energy_kwh

# Ensure datetime
X["date"] = pd.to_datetime(X["date"])
y["date"] = pd.to_datetime(y["date"])

print("Features shape:", X.shape)
print("Targets shape :", y.shape)
display(X.head(3))
display(y.head(3))


Features shape: (212073, 14)
Targets shape : (212073, 3)


,ZoneID,date,energy_kwh,year,month,doy,dow,is_weekend,roll7_mean,roll7_std,roll7_z,roll30_mean,clim_month_kwh,anom_month_kwh
0,BLR-0001,2023-01-02,57.866284,2023,1,2,0,False,NaN,NaN,NaN,NaN,50.815418,7.050867
1,BLR-0001,2023-01-03,53.828780,2023,1,3,1,False,NaN,NaN,NaN,NaN,50.815418,3.013362
2,BLR-0001,2023-01-04,54.480626,2023,1,4,2,False,55.391897,1.769778,-0.514907,NaN,50.815418,3.665208


,ZoneID,date,energy_kwh
0,BLR-0001,2023-01-02,57.866284
1,BLR-0001,2023-01-03,53.828780
2,BLR-0001,2023-01-04,54.480626


In [13]:
# Cell 3 — Merge features with target on (ZoneID, date)
# Keep only rows where target is available for now
df = (X.merge(y, on=["ZoneID","date"], how="inner")
        .sort_values(["ZoneID","date"])
        .reset_index(drop=True))

# Standardize target column name
if "energy_kwh_y" in df.columns:
    df = df.rename(columns={"energy_kwh_y": "energy_kwh"})
if "energy_kwh_x" in df.columns:
    df = df.drop(columns=["energy_kwh_x"])  # features version, redundant

print(df.shape)
df.head(5)


(212073, 14)


,ZoneID,date,year,month,doy,dow,is_weekend,roll7_mean,roll7_std,roll7_z,roll30_mean,clim_month_kwh,anom_month_kwh,energy_kwh
0,BLR-0001,2023-01-02,2023,1,2,0,False,NaN,NaN,NaN,NaN,50.815418,7.050867,57.866284
1,BLR-0001,2023-01-03,2023,1,3,1,False,NaN,NaN,NaN,NaN,50.815418,3.013362,53.828780
2,BLR-0001,2023-01-04,2023,1,4,2,False,55.391897,1.769778,-0.514907,NaN,50.815418,3.665208,54.480626
3,BLR-0001,2023-01-05,2023,1,5,3,False,52.250817,5.652277,-1.667158,NaN,50.815418,-7.987840,42.827577
4,BLR-0001,2023-01-06,2023,1,6,4,False,51.574389,5.233431,-0.517005,NaN,50.815418,-1.946738,48.868679


In [14]:
# Cell 4 — Basic sanity checks & type cleanups
# Drop exact duplicate rows (if any)
before = len(df)
df = df.drop_duplicates(subset=["ZoneID","date"])
print(f"Deduped rows: {before - len(df)}")

# Sort and set helpful keys
df = df.sort_values(["ZoneID","date"]).reset_index(drop=True)

# Identify numeric features (auto) — exclude keys and target
KEYS = ["ZoneID","date"]
TARGET = "energy_kwh"
non_feature_cols = set(KEYS + [TARGET])
numeric_cols = [c for c in df.columns 
                if c not in non_feature_cols and pd.api.types.is_numeric_dtype(df[c])]
print("Auto-detected numeric feature columns:", numeric_cols[:15], "..." if len(numeric_cols)>15 else "")


Deduped rows: 0
Auto-detected numeric feature columns: ['year', 'month', 'doy', 'dow', 'is_weekend', 'roll7_mean', 'roll7_std', 'roll7_z', 'roll30_mean', 'clim_month_kwh', 'anom_month_kwh'] 


In [15]:
# Cell 5 — Add target lags & rolling stats (per ZoneID)
# These help baselines and supervised models alike.
def add_lags_rolls(g):
    g = g.sort_values("date").copy()
    g["y_lag1"]  = g[TARGET].shift(1)
    g["y_lag7"]  = g[TARGET].shift(7)
    g["y_lag14"] = g[TARGET].shift(14)
    g["y_mean_7d"] = g[TARGET].shift(1).rolling(7, min_periods=3).mean()
    g["y_std_7d"]  = g[TARGET].shift(1).rolling(7, min_periods=3).std()
    g["y_min_7d"]  = g[TARGET].shift(1).rolling(7, min_periods=3).min()
    g["y_max_7d"]  = g[TARGET].shift(1).rolling(7, min_periods=3).max()
    return g

df = df.groupby("ZoneID", group_keys=False).apply(add_lags_rolls)

# Seasonality features
df["month"] = df["date"].dt.month
df["dow"]   = df["date"].dt.dayofweek
df["doy"]   = df["date"].dt.dayofyear
# cyclical encodings for annual seasonality
df["sin_doy"] = np.sin(2*np.pi*df["doy"]/365.25)
df["cos_doy"] = np.cos(2*np.pi*df["doy"]/365.25)

print("Added lag/rolling + seasonality features.")
df.head(5)


Added lag/rolling + seasonality features.


/var/folders/dy/m8k3w5n10rg7xhthbf5pxwyr0000gn/T/ipykernel_44108/3427088874.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("ZoneID", group_keys=False).apply(add_lags_rolls)


,ZoneID,date,year,month,doy,dow,is_weekend,roll7_mean,roll7_std,roll7_z,...,energy_kwh,y_lag1,y_lag7,y_lag14,y_mean_7d,y_std_7d,y_min_7d,y_max_7d,sin_doy,cos_doy
0,BLR-0001,2023-01-02,2023,1,2,0,False,NaN,NaN,NaN,...,57.866284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.034398,0.999408
1,BLR-0001,2023-01-03,2023,1,3,1,False,NaN,NaN,NaN,...,53.828780,57.866284,NaN,NaN,NaN,NaN,NaN,NaN,0.051584,0.998669
2,BLR-0001,2023-01-04,2023,1,4,2,False,55.391897,1.769778,-0.514907,...,54.480626,53.828780,NaN,NaN,NaN,NaN,NaN,NaN,0.068755,0.997634
3,BLR-0001,2023-01-05,2023,1,5,3,False,52.250817,5.652277,-1.667158,...,42.827577,54.480626,NaN,NaN,55.391897,2.167527,53.828780,57.866284,0.085906,0.996303
4,BLR-0001,2023-01-06,2023,1,6,4,False,51.574389,5.233431,-0.517005,...,48.868679,42.827577,NaN,NaN,52.250817,6.526687,42.827577,57.866284,0.103031,0.994678


In [16]:
# Cell 6 — Finalize feature list (auto + engineered)
# Recompute numeric columns now that we've added engineered features
non_feature_cols = set(KEYS + [TARGET])
FEATS = [c for c in df.columns 
         if c not in non_feature_cols and pd.api.types.is_numeric_dtype(df[c])]

print(f"Total candidate features: {len(FEATS)}")
# peek a few
FEATS[:20]


Total candidate features: 20


['year',
 'month',
 'doy',
 'dow',
 'is_weekend',
 'roll7_mean',
 'roll7_std',
 'roll7_z',
 'roll30_mean',
 'clim_month_kwh',
 'anom_month_kwh',
 'y_lag1',
 'y_lag7',
 'y_lag14',
 'y_mean_7d',
 'y_std_7d',
 'y_min_7d',
 'y_max_7d',
 'sin_doy',
 'cos_doy']

In [17]:
# Cell 7 — Define time-based split (holdout = last 60 days globally)
last_date = df["date"].max()
holdout_start = last_date - pd.Timedelta(days=60)

train_df = df[df["date"] < holdout_start].copy()
test_df  = df[df["date"] >= holdout_start].copy()

print("Date range:", df["date"].min().date(), "→", df["date"].max().date())
print("Holdout start:", holdout_start.date())
print("Train size:", train_df.shape, "Test size:", test_df.shape)


Date range: 2023-01-02 → 2025-08-09
Holdout start: 2025-06-10
Train size: (198470, 23) Test size: (13603, 23)


In [26]:
# Cell 8 — Baseline 1 (fixed): Persistence y_hat_t = y_(t-1) within same ZoneID

# 0) Safety: sorted + normalized keys
df["date"] = pd.to_datetime(df["date"]).dt.normalize()
df = df.sort_values(["ZoneID","date"]).reset_index(drop=True)

# 1) Build yesterday's value per ZoneID on the FULL SERIES (then we'll slice test)
df["y_hat_persist"] = df.groupby("ZoneID")[TARGET].shift(1)

# 2) Slice holdout and DROP rows where we don't have a lag (first day per zone in holdout)
test_eval = df[df["date"] >= holdout_start].copy()
test_eval = test_eval.dropna(subset=["y_hat_persist"])

# 3) Metrics (no forward-fill — that can silently create zeros)
def mape(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.clip(np.abs(y_true), 1e-6, None)
    return float(np.mean(np.abs(y_true - y_pred) / denom) * 100)

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((np.asarray(y_true) - np.asarray(y_pred))**2)))

y_true = test_eval[TARGET].values
y_pred = test_eval["y_hat_persist"].values

mae_persist  = float(np.mean(np.abs(y_true - y_pred)))
mape_persist = mape(y_true, y_pred)
rmse_persist = rmse(y_true, y_pred)

print("Baseline — Persistence (fixed)")
print(" Evaluated rows:", len(test_eval))
print(" MAE :", round(mae_persist, 3))
print(" MAPE:", round(mape_persist, 3), "%")
print(" RMSE:", round(rmse_persist, 3))



Baseline — Persistence (fixed)
 Evaluated rows: 13603
 MAE : 0.0
 MAPE: 0.0 %
 RMSE: 0.0


In [ ]:
# Cell 9 — Baseline 2: Climatology (zone × month mean from TRAIN only)
climo = (train_df.assign(month=train_df["date"].dt.month)
         .groupby(["ZoneID","month"])[TARGET]
         .mean()
         .rename("climo")
         .reset_index())

test_m = test_df[["ZoneID","date",TARGET]].copy()
test_m["month"] = test_m["date"].dt.month
test_m = test_m.merge(climo, on=["ZoneID","month"], how="left")

# If a zone-month combo never appeared in train, fallback to zone mean
zone_mean = train_df.groupby("ZoneID")[TARGET].mean().rename("zone_mean").reset_index()
test_m = test_m.merge(zone_mean, on="ZoneID", how="left")
test_m["climo"] = test_m["climo"].fillna(test_m["zone_mean"])

mae_climo  = float(np.mean(np.abs(test_m[TARGET].values - test_m["climo"].values)))
mape_climo = mape(test_m[TARGET].values, test_m["climo"].values)
rmse_climo = rmse(test_m[TARGET].values, test_m["climo"].values)

print("Baseline — Climatology (zone×month)")
print(" MAE :", round(mae_climo,3))
print(" MAPE:", round(mape_climo,3), "%")
print(" RMSE:", round(rmse_climo,3))


Baseline — Climatology (zone×month)
 MAE : 35.788
 MAPE: 3578757463.746 %
 RMSE: 36.04


In [20]:
# Cell 10 — Baseline metrics dataframe (overall + per-zone)
# Overall
overall = pd.DataFrame([
    {"baseline":"persistence","scope":"overall","MAE":mae_persist,"MAPE":mape_persist,"RMSE":rmse_persist},
    {"baseline":"climatology","scope":"overall","MAE":mae_climo, "MAPE":mape_climo, "RMSE":rmse_climo},
])

# Per-zone
# Persistence per-zone
test_df_idxed = test_df.set_index(["ZoneID","date"])
persist_df = test_df_idxed[[TARGET]].copy()
persist_df["y_hat"] = baseline_persist.values
def zone_metrics(g):
    return pd.Series({
        "MAE": float(np.mean(np.abs(g[TARGET] - g["y_hat"]))),
        "MAPE": mape(g[TARGET], g["y_hat"]),
        "RMSE": rmse(g[TARGET], g["y_hat"])
    })
perzone_persist = persist_df.groupby(level=0).apply(zone_metrics).reset_index(names="ZoneID")
perzone_persist["baseline"] = "persistence"

# Climatology per-zone
climo_df = test_m.copy()
climo_df = climo_df.rename(columns={"climo":"y_hat"})
perzone_climo = (climo_df
                 .groupby("ZoneID")
                 .apply(lambda g: pd.Series({
                     "MAE": float(np.mean(np.abs(g[TARGET] - g["y_hat"]))),
                     "MAPE": mape(g[TARGET], g["y_hat"]),
                     "RMSE": rmse(g[TARGET], g["y_hat"])
                 }))
                 .reset_index())
perzone_climo["baseline"] = "climatology"

perzone = pd.concat([perzone_persist, perzone_climo], ignore_index=True)
perzone["scope"] = "per-zone"

baseline_metrics = pd.concat([overall, perzone], ignore_index=True)
baseline_metrics.head(10)


/var/folders/dy/m8k3w5n10rg7xhthbf5pxwyr0000gn/T/ipykernel_44108/979708315.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,baseline,scope,MAE,MAPE,RMSE,ZoneID
0,persistence,overall,0.000000,0.000000e+00,0.000000,NaN
1,climatology,overall,35.787575,3.578757e+09,36.040122,NaN
2,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0001
3,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0002
4,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0003
5,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0004
6,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0005
7,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0006
8,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0007
9,persistence,per-zone,0.000000,0.000000e+00,0.000000,BLR-0008


In [22]:
# Cell 11 — Save dataset + baselines
# Keep a clean dataset for Phase 6 models
phase5_dataset = df[["ZoneID","date","energy_kwh"] + FEATS].copy()
phase5_dataset.to_parquet(OUT_DATASET, index=False)

baseline_metrics.to_csv(OUT_BASELINES, index=False)

print("Saved:")
print(" -", OUT_DATASET)
print(" -", OUT_BASELINES)


Saved:
 - data/processed/phase5_dataset.parquet
 - data/processed/phase5_baselines.csv
